In [1]:
# 1. Imports nécessaires
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, classification_report

# 2. Chargement des données
# 'cleaned_description' (le texte) et 'main_category' (la cible/target).
file_path = '../Projet6/Data/df_desc_cleaned.csv'
df = pd.read_csv(file_path)
df.head(10)

C:\Users\Thomas\Documents\Data_Science\Projet8\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,description,main_category,cleaned_description
0,Key Features of Elegance Polyester Multicolor ...,Home Furnishing,eleg polyest multicolor abstract eyelet door c...
1,Specifications of Sathiyas Cotton Bath Towel (...,Baby Care,sathiya cotton bath towel bath towel yellow ba...
2,Key Features of Eurospa Cotton Terry Face Towe...,Baby Care,eurospa cotton terri face towel small inch gsm...
3,Key Features of SANTOSH ROYAL FASHION Cotton P...,Home Furnishing,santosh royal fashion cotton print king doubl ...
4,Key Features of Jaipur Print Cotton Floral Kin...,Home Furnishing,jaipur print cotton floral king doubl bedsheet...
5,Maserati Time R8851116001 Analog Watch - For ...,Watches,maserati time analog watch boy maserati time a...
6,"Camerii WM64 Elegance Analog Watch - For Men,...",Watches,camerii eleg analog watch men boy camerii eleg...
7,T STAR UFT-TSW-005-BK-BR Analog Watch - For B...,Watches,star analog watch boy whether way work travel ...
8,"Alfajr WY16B Youth Digital Watch - For Men, B...",Watches,alfajr youth digit watch men boy alfajr youth ...
9,TAG Heuer CAU1116.BA0858 Formula 1 Analog Watc...,Watches,tag heuer formula analog watch boy men tag heu...


In [ ]:
# 1. Création des embeddings avec Sentence-Transformer
print("Étape 1: Création des embeddings...")
model_embed = SentenceTransformer('all-MiniLM-L6-v2')
X_embeddings = model_embed.encode(df['cleaned_description'].tolist(), show_progress_bar=True)
y = df['main_category']

# 2. Séparation des données
print("\nÉtape 2: Séparation des données en train/test...")
X_train, X_test, y_train, y_test = train_test_split(
    X_embeddings, y, test_size=0.2, random_state=42, stratify=y
)
print(f"Taille de l'entraînement : {X_train.shape[0]} échantillons")
print(f"Taille du test : {X_test.shape[0]} échantillons")

# 3. Entraînement du modèle de référence (Régression Logistique)
print("\nÉtape 3: Entraînement du modèle de référence (Régression Logistique)...")
baseline_model = LogisticRegression(max_iter=1000)
baseline_model.fit(X_train, y_train)

# 4. Évaluation et récupération des métriques
print("\nÉtape 4: Évaluation du modèle de référence...")
y_pred = baseline_model.predict(X_test)

# Calcul et affichage des métriques complètes
f1 = f1_score(y_test, y_pred, average='weighted')
report = classification_report(y_test, y_pred)

print("\n--- Performance de la Baseline ---")
print(f"Modèle : Régression Logistique sur embeddings Sentence-Transformer")
print(f"F1-Score (pondéré) : {f1:.4f}")
print("\n--- Rapport de classification détaillé ---")
print(report)

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from datasets import Dataset

# Split des données textuelles originales
X_train_text, X_test_text, y_train_text, y_test_text = train_test_split(
    df['cleaned_description'], df['main_category'], test_size=0.2, random_state=42, stratify=df['main_category']
)

# Encodage des labels
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train_text)
y_test_encoded = le.transform(y_test_text)

# Création du DataFrame d'entraînement complet
df_train = pd.DataFrame({'text': X_train_text, 'label': y_train_encoded})

# Échantillonnage Few-Shot
df_train_few_shot = df_train.groupby('label', group_keys=False).sample(n=16, random_state=42)

# Réinitialisation de l'index du jeu d'entraînement
df_train_few_shot = df_train_few_shot.reset_index(drop=True)

# Création du DataFrame de test
df_test = pd.DataFrame({'text': X_test_text, 'label': y_test_encoded})

# Réinitialisation de l'index du jeu de test
df_test = df_test.reset_index(drop=True)

# Création des Datasets
train_dataset = Dataset.from_pandas(df_train_few_shot)
test_dataset = Dataset.from_pandas(df_test)

# Vérification finale
print("Colonnes du jeu d'entraînement SetFit:", train_dataset.column_names)
print("Colonnes du jeu de test SetFit:", test_dataset.column_names)

Colonnes du jeu d'entraînement SetFit: ['text', 'label']
Colonnes du jeu de test SetFit: ['text', 'label']


In [11]:
from setfit import SetFitModel, Trainer, TrainingArguments
from sklearn.metrics import f1_score
from datasets import Dataset

# 1. Préparation des données (cette partie ne change pas)
# ... (ton code pour créer train_dataset et test_dataset) ...


# 2. Charger le modèle pré-entraîné (ne change pas)
model = SetFitModel.from_pretrained(
    "sentence-transformers/all-MiniLM-L6-v2",
    num_labels=len(df['main_category'].unique())
)


# --- ✨ LA CORRECTION EST ICI ✨ ---

# 3. Définir les arguments d'entraînement
# C'est la nouvelle façon de faire, plus modulaire
args = TrainingArguments(
    output_dir="setfit_model",      # Un dossier pour sauvegarder les résultats
    num_epochs=1,             # Le nombre d'époques pour l'entraînement contrastif
    batch_size=16,
    evaluation_strategy="epoch",    # Évaluer à la fin de chaque époque
)

# 4. Créer une fonction pour calculer les métriques
# Le nouveau Trainer attend une fonction qui prend un tuple (prédictions, étiquettes)
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    return {
        "f1": f1_score(labels, predictions, average="weighted")
    }

# 5. Créer le Trainer (la nouvelle classe recommandée)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    column_mapping={"text": "text", "label": "label"},
    metric=compute_metrics
)

# 6. Lancer l'entraînement
trainer.train()

# 7. Évaluer
metrics = trainer.evaluate()

print("\n--- Performance de SetFit (Few-Shot) ---")
print(metrics)

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
The `evaluation_strategy` argument is deprecated and will be removed in a future version. Please use `eval_strategy` instead.
Applying column mapping to the training dataset
Applying column mapping to the evaluation dataset
Map: 100%|██████████| 112/112 [00:00<00:00, 18658.38 examples/s]
***** Running training *****
  Num unique pairs = 10752
  Batch size = 16
  Num epochs = 1
C:\Users\Thomas\Documents\Data_Science\Projet8\.venv\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 